* Nama : Alif Gala Buana
* Kelas : Bootcamp Data Science 27





## Import Library

In [7]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from imblearn.over_sampling import SMOTE

from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, recall_score

## Load dataset

In [8]:
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn .csv')
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors = 'coerce')
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
df['Churn'].value_counts(normalize=True)

,proportion
Churn,
No,0.73463
Yes,0.26537


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


## Data Preprocessing

In [11]:
data =pd.get_dummies(data=df,columns=['gender', 'Partner', 'Dependents',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod'],drop_first=True)

data['Churn'] = data['Churn'].replace("No", 0).replace("Yes", 1)
data.head()

,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,0,1,29.85,29.85,0,False,True,False,False,...,False,False,False,False,False,False,True,False,True,False
1,5575-GNVDE,0,34,56.95,1889.50,0,True,False,False,True,...,False,False,False,False,True,False,False,False,False,True
2,3668-QPYBK,0,2,53.85,108.15,1,True,False,False,True,...,False,False,False,False,False,False,True,False,False,True
3,7795-CFOCW,0,45,42.30,1840.75,0,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,9237-HQITU,0,2,70.70,151.65,1,False,False,False,True,...,False,False,False,False,False,False,True,False,True,False


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   customerID                             7043 non-null   object 
 1   SeniorCitizen                          7043 non-null   int64  
 2   tenure                                 7043 non-null   int64  
 3   MonthlyCharges                         7043 non-null   float64
 4   TotalCharges                           7032 non-null   float64
 5   Churn                                  7043 non-null   int64  
 6   gender_Male                            7043 non-null   bool   
 7   Partner_Yes                            7043 non-null   bool   
 8   Dependents_Yes                         7043 non-null   bool   
 9   PhoneService_Yes                       7043 non-null   bool   
 10  MultipleLines_No phone service         7043 non-null   bool   
 11  Mult

In [13]:
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
data.TotalCharges = imputer.fit_transform(data["TotalCharges"].values.reshape(-1, 1))

In [14]:
data['Churn'].value_counts(normalize=True)

,proportion
Churn,
0,0.73463
1,0.26537


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   customerID                             7043 non-null   object 
 1   SeniorCitizen                          7043 non-null   int64  
 2   tenure                                 7043 non-null   int64  
 3   MonthlyCharges                         7043 non-null   float64
 4   TotalCharges                           7043 non-null   float64
 5   Churn                                  7043 non-null   int64  
 6   gender_Male                            7043 non-null   bool   
 7   Partner_Yes                            7043 non-null   bool   
 8   Dependents_Yes                         7043 non-null   bool   
 9   PhoneService_Yes                       7043 non-null   bool   
 10  MultipleLines_No phone service         7043 non-null   bool   
 11  Mult

In [16]:
sm = SMOTE(random_state=42)
X = data.drop(['Churn','customerID'],axis = 1)
Y = data['Churn']
X_sm, y_sm = sm.fit_resample(X, Y)

print(f'''Shape of X before SMOTE: {X.shape}
Shape of X after SMOTE: {X_sm.shape}''')

print('\nBalance of positive and negative classes (%):')
y_sm.value_counts(normalize=True) * 100

Shape of X before SMOTE: (7043, 30)
Shape of X after SMOTE: (10348, 30)

Balance of positive and negative classes (%):


,proportion
Churn,
0,50.0
1,50.0


## Modeling

* splitting data yang sebelum di SMOTE

In [17]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=0.25, random_state=42)

* splitting data yang sudah di SMOTE

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.25, random_state=42)

### Inisiasi Model-model yang digunakan

In [29]:
model = XGBClassifier(random_state=42)

df = pd.DataFrame(columns = ['Method', 'Accuracy','Recall','Precision'])
df

,Method,Accuracy,Recall,Precision


## Evaluation

### Evaluasi Model untuk data yang belum di SMOTE

In [32]:
    model.fit(X_train2, y_train2)
    preds2 = model.predict(X_test2)

    method = str(type(model)).split('.')[-1][:-2]

    # acc
    rec = recall_score(y_test2, preds2)
    prec = precision_score(y_test2, preds2)
    acc = accuracy_score(y_test2, preds2)
    new_row = pd.DataFrame([{'Method': method, 'Accuracy': acc, 'Recall':rec,'Precision':prec}])

# Menggabungkan DataFrame lama dengan baris baru
df2 = pd.concat([df, new_row], ignore_index=True)

In [21]:
df2

,Method,Accuracy,Recall,Precision
0,XGBClassifier,0.785349,0.501044,0.633245


* Akurasi cukup baik, tetapi recall rendah menunjukkan model kesulitan mendeteksi kelas minoritas.
* Precision lebih tinggi dari recall, menunjukkan prediksi positif lebih sering benar, namun banyak kelas minoritas yang tidak terdeteksi.

### Evaluasi Model untuk data sesudah di SMOTE

In [31]:

model.fit(X_train, y_train)
preds = model.predict(X_test)

method = str(type(model)).split('.')[-1][:-2]

# acc
rec = recall_score(y_test, preds)
prec = precision_score(y_test, preds)
acc = accuracy_score(y_test, preds)
new_row = pd.DataFrame([{'Method': method, 'Accuracy': acc, 'Recall':rec,'Precision':prec}])

# Menggabungkan DataFrame lama dengan baris baru
df1 = pd.concat([df, new_row], ignore_index=True)

In [23]:
df1

,Method,Accuracy,Recall,Precision
0,XGBClassifier,0.851179,0.857254,0.846095


* Akurasi meningkat setelah SMOTE.
* Recall meningkat signifikan, menunjukkan model lebih baik dalam mendeteksi kelas minoritas.
* Precision juga meningkat, membuat prediksi lebih akurat secara keseluruhan.

## Kesimpulan

* SMOTE berhasil meningkatkan kinerja model, terutama dalam mendeteksi kelas minoritas, tanpa mengorbankan akurasi.
* Model yang terbaik digunakan adalah XGBClassifier

In [33]:
# classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.86      0.85      0.85      1298
           1       0.85      0.86      0.85      1289

    accuracy                           0.85      2587
   macro avg       0.85      0.85      0.85      2587
weighted avg       0.85      0.85      0.85      2587



In [34]:
# Simpan model dalam format JSON
model.save_model('trained_model.json')
print("Model berhasil disimpan sebagai JSON.")

Model berhasil disimpan sebagai JSON.
